# Project 2
## Step 1: Explore data

##### Load your data in using Pandas and start to explore. Save all of your early exploration code here and include in your final submission.

In [1]:
#Importing packages 
import pandas as pd, numpy as np, seaborn as sns, matplotlib.pyplot as plt, datetime
from datetime import timedelta
%matplotlib inline

##### Write a brief description of your data, data dictionary, and any interesting observations you've made thus far. 

In [2]:
#read the data from csv and assign it to a dataframe
bd_original = pd.read_csv("../assets/billboard.csv", header=0)

In [3]:
#Wanted to verify what is created
type(bd_original)

pandas.core.frame.DataFrame

In [ ]:
#checking how many rows and columns are here.
#bd_original.columns
#317 rows, 83 columns 

In [ ]:
#bd_original.info()


#getting some more info about the DataFrame
##Initial thoughts on what I need to clean: 
#change title artist.inverted to artist
#change title date.entered to date_entered
#change title date.peaked to date_peaked
#change date_entered and date_peaked types to datetime
#strip 'x', 'st' from week titles
#replace '.' with '_' in week titles


In [ ]:
#bd_original.head()

In [ ]:
#using describe to get more detailed info on the data and to understand how it differs from info()
#Got 'RuntimeWarning: Invalid value encountered in percentile' error when running describe. I think it has to do with 
#some of the columns being all nan

#bd_original.describe(include='all')

##Initial notes and thoughts on what the numbers might show us from describe: 
#top artist: Jay-Z
#top track: Where I Wanna Be
#top genre: Rock
#unique: 228 out of 317 artists
#unique: 68 out of 317 date_entered
#unique: 53 out of 317 date_peaked
#avg rank on week 1: 79.958991
#min rank on week 1: 15.000000
#max rank on week 1: 100.000000


In [ ]:
# bd_original.rename(columns=lambda x: x.replace('x', '').replace('st', '').replace('nd', '').replace('rd', '').replace('th', '').replace('.week', ''), inplace=True)

In [4]:
bd_original.head()

,year,artist.inverted,track,time,genre,date.entered,date.peaked,x1st.week,x2nd.week,x3rd.week,...,x67th.week,x68th.week,x69th.week,x70th.week,x71st.week,x72nd.week,x73rd.week,x74th.week,x75th.week,x76th.week
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,78,63.0,49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,15,8.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,71,48.0,43.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,41,23.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,57,47.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Step 2: Clean data

##### Do some rudimentary cleaning. Rename any columns that are poorly named, shorten any strings that are too long, and check for missing values (replace them and explain rationale if it makes sense).

In [60]:
#rename id columns
bd_original.rename(columns={'year':'year_released','artist.inverted':'artist', 'date.entered':'date_entered', 'date.peaked':'date_peaked'}, inplace=True)

In [61]:
#change date formats
bd_original['date_entered'] = pd.to_datetime(bd_original['date_entered'])
bd_original['date_peaked'] = pd.to_datetime(bd_original['date_peaked'])

In [62]:
bd_original['weeks_on_chart'] = bd_original.ix[:,7:].notnull().sum(axis=1)

In [63]:
bd_original['days_to_peak'] = bd_original['date_peaked'] - bd_original['date_entered']

In [64]:
bd_original.head()

,year_released,artist,track,time,genre,date_entered,date_peaked,x1st.week,x2nd.week,x3rd.week,...,x69th.week,x70th.week,x71st.week,x72nd.week,x73rd.week,x74th.week,x75th.week,x76th.week,weeks_on_chart,days_to_peak
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,78,63.0,49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30,56 days
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,15,8.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28,56 days
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,71,48.0,43.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35,98 days
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,41,23.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,35 days
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,57,47.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23,70 days


In [10]:
#drop entire column if all data is NaN in column
##params:
#axis='1' drops the entire column instead of dropping an entire row
#how='all' only drops columns where all the rows for that column are nan

bd_nan_dropped = bd_original.dropna(how='all', axis=1)

#verifying shape to make sure number of columns dropped, and rows are uneffected
#new rows = 317
#new columns = 72
bd_nan_dropped.shape

(317, 74)

In [11]:
bd_nan_dropped.head()

,year,artist,track,time,genre,date_entered,date_peaked,x1st.week,x2nd.week,x3rd.week,...,x58th.week,x59th.week,x60th.week,x61st.week,x62nd.week,x63rd.week,x64th.week,x65th.week,weeks_on_chart,days_to_peak
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,78,63.0,49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28,56 days
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,15,8.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,56 days
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,71,48.0,43.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33,98 days
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,41,23.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24,35 days
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,57,47.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21,70 days


In [12]:
bd_nan_dropped.describe(include='all')

//anaconda/envs/py27/lib/python2.7/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,year,artist,track,time,genre,date_entered,date_peaked,x1st.week,x2nd.week,x3rd.week,...,x58th.week,x59th.week,x60th.week,x61st.week,x62nd.week,x63rd.week,x64th.week,x65th.week,weeks_on_chart,days_to_peak
count,317.0,317,317,317,317,317,317,317.000000,312.000000,307.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.0,1.0,317.000000,317
unique,NaN,228,316,126,10,68,53,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,Jay-Z,Where I Wanna Be,3:50,Rock,2000-01-29 00:00:00,2000-12-02 00:00:00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,5,2,9,137,11,17,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
first,NaN,NaN,NaN,NaN,NaN,1999-06-05 00:00:00,2000-01-01 00:00:00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,NaN,2000-12-30 00:00:00,2000-12-30 00:00:00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,2000.0,NaN,NaN,NaN,NaN,NaN,NaN,79.958991,71.173077,65.045603,...,31.500000,35.500000,38.000000,42.500000,44.500000,47.500000,50.0,49.0,16.741325,52 days 05:54:19.305993
std,0.0,NaN,NaN,NaN,NaN,NaN,NaN,14.686865,18.200443,20.752302,...,0.707107,4.949747,1.414214,0.707107,3.535534,3.535534,0.0,NaN,9.083785,40 days 20:49:20.696756
min,2000.0,NaN,NaN,NaN,NaN,NaN,NaN,15.000000,8.000000,6.000000,...,31.000000,32.000000,37.000000,42.000000,42.000000,45.000000,50.0,49.0,1.000000,0 days 00:00:00
25%,2000.0,NaN,NaN,NaN,NaN,NaN,NaN,74.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.000000,21 days 00:00:00


In [ ]:
#bd_year_dropped.head()
#What I've completed so far. Just creating a checkpoint for myself to brainstorm next steps:
#change title artist.inverted to artist - DONE
#change title date.entered to date_entered - DONE
#change title date.peaked to date_peaked - DONE
#change date_entered and date_peaked types to datetime - DONE
#strip 'x', 'st' from week titles - DONE
#replace '.' with '_' in week titles - DONE

In [ ]:
#WHAT NEXT?
##From this current data, what new data can I extrapulate and add as a new column to the cleaned dataset?
#How many weeks did each track stay on the chart?
#How long did it take each track to reach it's peak?

In [ ]:
#How many weeks did each track stay on the chart?
#Creating new column 'weeks_on_chart' to show how long each track was on the chart

##### Using Pandas' built in `melt` function, pivot the weekly ranking data to be long rather than wide. As a result, you will have removed the 72 'week' columns and replace it with two: Week and Ranking. There will now be multiple entries for each song, one for each week on the Billboard rankings.

In [13]:
bd_melt = pd.melt(bd_nan_dropped, id_vars=['year', 'artist', 'track', 'time', 'genre', 'date_entered', 'date_peaked', 'weeks_on_chart', 'days_to_peak'], value_vars=list(bd_nan_dropped.columns.values[7:-2]), var_name='Week', value_name='Ranking')

In [14]:
bd_melt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20605 entries, 0 to 20604
Data columns (total 11 columns):
year              20605 non-null int64
artist            20605 non-null object
track             20605 non-null object
time              20605 non-null object
genre             20605 non-null object
date_entered      20605 non-null datetime64[ns]
date_peaked       20605 non-null datetime64[ns]
weeks_on_chart    20605 non-null int64
days_to_peak      20605 non-null timedelta64[ns]
Week              20605 non-null object
Ranking           5307 non-null float64
dtypes: datetime64[ns](2), float64(1), int64(2), object(5), timedelta64[ns](1)
memory usage: 1.7+ MB


In [15]:
bd_melt['Week'] = bd_melt['Week'].map(lambda x: x.replace('x', '').replace('st', '').replace('nd', '').replace('rd', '').replace('th', '').replace('.week', ''))

In [16]:
bd_melt.head()

,year,artist,track,time,genre,date_entered,date_peaked,weeks_on_chart,days_to_peak,Week,Ranking
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,28,56 days,1,78.0
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,26,56 days,1,15.0
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,33,98 days,1,71.0
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,24,35 days,1,41.0
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,21,70 days,1,57.0


In [17]:
bd_melt['Week'] = bd_melt['Week'].astype(int)

In [18]:
week_date = []

for index, date in enumerate(bd_melt['date_entered']):
    d = date
    w = timedelta(days=((int(bd_melt['Week'][index])-1)*7))
    week_date.insert(index, d + w)
    
bd_melt['week_date'] = week_date

In [59]:
bd_melt.head()

,year,artist,track,time,genre,date_entered,date_peaked,weeks_on_chart,days_to_peak,Week,Ranking,week_date,rank_on_chart
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,28,56 days,1,78.0,2000-09-23,23.0
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,26,56 days,1,15.0,2000-02-12,86.0
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,33,98 days,1,71.0,1999-10-23,30.0
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,24,35 days,1,41.0,2000-08-12,60.0
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,21,70 days,1,57.0,2000-08-05,44.0


In [56]:
bd_melt.dtypes

year                        int64
artist                     object
track                      object
time                       object
genre                    category
date_entered       datetime64[ns]
date_peaked        datetime64[ns]
weeks_on_chart              int64
days_to_peak      timedelta64[ns]
Week                        int64
Ranking                   float64
week_date          datetime64[ns]
rank_on_chart             float64
dtype: object

In [55]:
bd_melt['genre'] = bd_melt['genre'].astype('category')

In [29]:
#add a date field to map the week number for each track to the week of the year it's actually on the track
bd_melt["rank_on_chart"] = [101 - x for x in bd_melt.Ranking]
bd_melt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20605 entries, 0 to 20604
Data columns (total 13 columns):
year              20605 non-null int64
artist            20605 non-null object
track             20605 non-null object
time              20605 non-null object
genre             20605 non-null object
date_entered      20605 non-null datetime64[ns]
date_peaked       20605 non-null datetime64[ns]
weeks_on_chart    20605 non-null int64
days_to_peak      20605 non-null timedelta64[ns]
Week              20605 non-null int64
Ranking           5307 non-null float64
week_date         20605 non-null datetime64[ns]
rank_on_chart     5307 non-null float64
dtypes: datetime64[ns](3), float64(2), int64(3), object(4), timedelta64[ns](1)
memory usage: 2.0+ MB


In [45]:
bd_track_counts = pd.pivot_table(bd_melt, index='track', aggfunc='count')

In [46]:
bd_track_counts.head()

,Ranking,Week,artist,date_entered,date_peaked,days_to_peak,genre,rank_on_chart,time,week_date,weeks_on_chart,year
track,,,,,,,,,,,,
(Hot S**t) Country Grammar,34,65,65,65,65,65,65,34,65,65,65,65
3 Little Words,9,65,65,65,65,65,65,9,65,65,65,65
911,19,65,65,65,65,65,65,19,65,65,65,65
A Country Boy Can Survive,3,65,65,65,65,65,65,3,65,65,65,65
A Little Gasoline,6,65,65,65,65,65,65,6,65,65,65,65


In [50]:
bd_melt_genre = pd.melt(bd_nan_dropped, id_vars=['genre', 'track'], value_vars=list(bd_nan_dropped.iloc[:, 7:-2]), var_name='Week', value_name='Ranking')

In [68]:
bd_genre_counts = pd.pivot_table(bd_melt_genre, index='track', columns='genre', aggfunc='count')

In [69]:
bd_genre_counts.head()

Week                                          \
genre                      Country Electronica Gospel Jazz Latin Pop   R&B   
track                                                                        
(Hot S**t) Country Grammar     NaN         NaN    NaN  NaN   NaN NaN   NaN   
3 Little Words                 NaN         NaN    NaN  NaN   NaN NaN  65.0   
911                            NaN         NaN    NaN  NaN   NaN NaN   NaN   
A Country Boy Can Survive     65.0         NaN    NaN  NaN   NaN NaN   NaN   
A Little Gasoline             65.0         NaN    NaN  NaN   NaN NaN   NaN   

                                             Ranking                          \
genre                        Rap Reggae Rock Country Electronica Gospel Jazz   
track                                                                          
(Hot S**t) Country Grammar  65.0    NaN  NaN     NaN         NaN    NaN  NaN   
3 Little Words               NaN    NaN  NaN     NaN         NaN    NaN  NaN   
911                         65.0    NaN  NaN     NaN         NaN    NaN  NaN   
A Country Boy Can Survive    NaN    NaN  NaN     3.0         NaN    NaN  NaN   
A Little Gasoline            NaN    NaN  NaN     6.0         NaN    NaN  NaN   

                                                             
genre                      Latin Pop  R&B   Rap Reggae Rock  
track                                                        
(Hot S**t) Country Grammar   NaN NaN  NaN  34.0    NaN  NaN  
3 Little Words               NaN NaN  9.0   NaN    NaN  NaN  
911                          NaN NaN  NaN  19.0    NaN  NaN  
A Country Boy Can Survive    NaN NaN  NaN   NaN    NaN  NaN  
A Little Gasoline            NaN NaN  NaN   NaN    NaN  NaN

In [67]:
bd_genre_counts['Country']

KeyError: 'Country'

## Step 3: Visualize your data.

##### Using BOTH a Tableau Dashboard and python, create visualizations that will provide context to your data. There is no minimum or maximum number of graphs you should generate, but there should be a clear and consistent story being told. Give insights to the distribution, statistics, and relationships of the data. 

## Step 4: Create a Problem Statement.

##### Having explored the data, come up with a problem statement that includes risks and assumptions for this data set. You can feel free to introduce data from any other source to support your problem statement, just be sure to provide a link to the origin of the data. Once again- be creative!

- Guide: http://www.ceptara.com/blog/how-to-write-problem-statement
- Kaggle example: https://www.kaggle.com/c/inria-bci-challenge
- Kaggle example: https://www.kaggle.com/c/axa-driver-telematics-analysis

## Step 5: Brainstorm your Approach.
##### In bullet-list form, provide a proposed approach for evaluating your problem statement. This can be somewhat high-level, but start to think about ways you can massage the data for maximum efficacy. 

## Step 6: Create a blog post with your code snippets and visualizations.
##### Data Science is a growing field, and the Tech industry thrives off of collaboration and sharing of knowledge. Blogging is a powerful means for pushing the needle forward in our field. Using your blogging platform of choice, create a site and post your process and results. Rather than writing a procedural text, imagine you're describing the data, visualizations, and conclusions you've arrived at to your peers.

### BONUS: LinkedIn has recognized you as a DS thought leader and asked you to pen a white paper (~500 words) on 'Best Practices for Clean Data'. This will be an opinion piece read by a wide (including semi-technical) audience, so be sure to back up your statements with real world examples or scenarios.

##### Hint: Look for online resources (articles, blogs, papers, youtube, podcasts, reddit) that will help you understand the challenges and implications of dealing with big data and missing data. This should be a personal reflection on everything you've learned this week, and the learning goals that have been set out for you going forward. 

Score | /24
----- | ------
Identify: Problem Statement / Hypothesis	| 		
Identify: Risks & Assumptions				| 
Acquire: Import Data Using Pandas			| 
Parse: Perform EDA Using Pandas				| 
Viz: Tableau Dashboard and Plots			| 
Mine: Determine Correlations				|
Refine: Evaluate Findings				    |
Present: Describe Results in Blog Post		|
Bonus! Present: Write a short White Paper	| /5